In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import RGBNirBioModule, RGBNirBioDataModule

dm = RGBNirBioDataModule(batch_size=512, num_workers=10, pin_memory=True)
dm.setup()

train: 1587395
val: 40080
test: 36421


In [5]:
model = RGBNirBioModule.load_from_checkpoint('./checkpoints/r18RGBNirBio2-val_loss=6.37801-epoch=2.ckpt')
model.hparams

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f8bb254aa60>
Traceback (most recent call last):
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1289, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f8bb254aa60>
Traceback (most recent call last):
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/juan/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1289, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdo

"backbone":         resnet18
"bio_dropout":      0.0
"bio_layers":       [100]
"datamodule":       {'batch_size': 512, 'num_workers': 10, 'pin_memory': True}
"early_stopping":   True
"num_bio":          27
"optimizer":        Adam
"optimizer_params": {'lr': 0.001}
"pretrained":       True
"trainer":          {'gpus': 1, 'max_epochs': 30, 'logger': <pytorch_lightning.loggers.wandb.WandbLogger object at 0x7f8b9ac710a0>, 'enable_checkpointing': True, 'overfit_batches': 0, 'deterministic': True, 'precision': 16, 'callbacks': [<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f8b9ac710d0>, <pytorch_lightning.callbacks.early_stopping.EarlyStopping object at 0x7f8b5bb49640>, <pytorch_lightning.callbacks.progress.tqdm_progress.TQDMProgressBar object at 0x7f8b5bb49670>, <pytorch_lightning.callbacks.model_summary.ModelSummary object at 0x7f8b5bb496a0>, <pytorch_lightning.callbacks.gradient_accumulation_scheduler.GradientAccumulationScheduler object at 0x7f8b5bb496d0>]}

In [6]:
from tqdm import tqdm
from src.GLC.metrics import top_30_error_rate
import numpy as np 

model.cuda(1)
dl = dm.val_dataloader()
accs = []
for batch in tqdm(dl):
    labels = batch['label']
    preds = model.predict(batch)
    accs.append(top_30_error_rate(labels, preds.cpu()))
np.mean(accs)

100%|██████████| 79/79 [00:46<00:00,  1.71it/s]


0.7152832718002813

In [7]:
from tqdm import tqdm

model.cuda(1)
dl = dm.test_dataloader()
labels, observations = [], []
for batch in tqdm(dl):
    preds = model.predict(batch)
    values, ixs = preds.topk(30)
    labels += [' '.join([str(i.item()) for i in ix]) for ix in ixs]
    observation_ids = batch['observation_id']
    observations += observation_ids.numpy().tolist()

100%|██████████| 72/72 [00:22<00:00,  3.25it/s]


In [8]:
import pandas as pd 

submission = pd.DataFrame({'Id': observations, 'Predicted': labels})
submission.to_csv('submission.csv', index=False)
submission.sample(5)

,Id,Predicted
31563,21548842,5053 5095 224 5189 3509 308 3072 5250 2020 527...
14789,10204734,167 15 321 625 353 4 937 988 864 414 273 1042 ...
19552,20250866,11563 5444 6016 2020 5064 6368 7196 6745 5068 ...
2148,10245491,29 1041 188 336 441 37 527 371 639 906 692 104...
10875,10570834,453 995 933 940 771 224 200 570 1231 533 988 7...


In [9]:
sample_submission = pd.read_csv('data/sample_submission.csv')
assert len(sample_submission) == len(submission)
sample_submission.sample(10)

,Id,Predicted
1912,10090563,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
4122,10194953,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2874,10137200,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
14822,10707009,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
14883,10709645,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
18491,20127100,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
16452,10784823,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
13513,10646253,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
28725,21244440,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
12334,10592213,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
